In [1]:
from pickletools import optimize
import torch, torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import os
from skimage import io, transform

from model import *
from utils import write_results, unique, bbox_iou
from dataset import filter_labels, DetectionDataset, Rescale, ToTensor, Normalise

plt.rcParams['figure.figsize'] = [15,15]

C:\Users\henry\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Rejig Custom Dataset

**Objective:** We want each label to be [x, y, w, h, obj?, c_1, c_2, ..., c_n]

### First find out n_unique classes/categories

In [2]:
# df = pd.read_json("D:/Datasets/bdd100k/labels/det_20/det_train.json")
# filtered_labels = filter_labels("D:/Datasets/bdd100k/labels/det_20/det_train.json")

{'bus', 'car', 'pedestrian', 'traffic light', 'traffic sign', 'truck', 'bicycle'}

In [3]:
classes = [
    'bus', 'car', 'pedestrian', 'traffic light', 
    'traffic sign', 'truck', 'bicycle']

## Load Dataset

### Testing manual pad transform

In [4]:
class Pad(object):
    """
    Add padding to image

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """
    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple)) # make sure output size is EITHER int or tuple
        self.output_size = output_size

    def __call__(self, sample):
        image, categories, bboxes = sample["image"], sample["categories"], sample["bboxes"]

        img_w, img_h = image.shape[1], image.shape[0]
        w, h = self.output_size

        # calculate new width and height
        new_w = int(img_w * min(w/img_w, h/img_h))
        new_h = int(img_h * min(w/img_w, h/img_h))
        resized_image = cv2.resize(image, (new_w,new_h), interpolation = cv2.INTER_CUBIC)
        
        canvas = np.full((self.output_size[1], self.output_size[0], 3), 128)

        canvas[(h-new_h)//2:(h-new_h)//2 + new_h,(w-new_w)//2:(w-new_w)//2 + new_w,  :] = resized_image

        return {"image": canvas, "categories": categories, "bboxes": bboxes}

In [5]:
filtered_labels = filter_labels("det_train_shortened.json")

train_data = DetectionDataset(
    label_dict=filtered_labels,
    root_dir='images/',
)
## load custom dataset + transforms
transformed_train_data = DetectionDataset(
    label_dict=filtered_labels,
    root_dir='images/',
    transform=transforms.Compose([
        Rescale(608),
        Normalise(0.5, 0.5),
        Pad((608,608)),
        ToTensor()
    ])
)

## dataloader
train_loader = DataLoader(
    transformed_train_data,
    batch_size=1,
    shuffle=True,
    num_workers=0
)

## Define Network

In [6]:
net = Net(cfgfile="cfg/model.cfg")

In [7]:
class TestNet(nn.Module):

    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)     # in 1 , out 6, 5x5 kernel
        self.conv2 = nn.Conv2d(6, 16, 5)    # in 6, out 16, 5x5 kernel

        self.pool = nn.MaxPool2d(2, 2)      # this time define own maxpooling
        
        # an affine op: y = Wx + b
        self.fc1 = nn.Linear(16*5*5, 120)   # 5x5 is image dim
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # max pooling over a (2,2) window
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dims except batch dim
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x) # no activation on final layer -> output
        return x
testnet = TestNet()

## Define Loss Function and Optimiser

In [8]:
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
criterion = ...

## Train Network

In [9]:
CUDA = torch.cuda.is_available()
for i, data in enumerate(train_loader):
    input_img, cat, bboxes = data.values()

    optimizer.zero_grad()
    #plt.imshow(input_img)
    #input_img = norm_with_padding(input_img, 608)
    outputs = net(Variable(input_img.float()), CUDA)
    

In [10]:
outputs[...,0] # select first bbox attr from all bbox tensors

tensor([[ 16.8364,  15.2165,  15.7572,  ..., 605.3627, 603.0162, 604.4326]])

## Troubleshooting bboxes

In [96]:
outputs = torch.load("1.pt")

In [97]:
outputs = outputs*(outputs[:,:,4] > 0.5).float().unsqueeze(2)

In [98]:
outputs.shape

torch.Size([1, 10647, 85])

In [99]:
box_corner = outputs.new(outputs.shape)
box_corner[:,:,0] = (outputs[:,:,0] - outputs[:,:,2]/2)
box_corner[:,:,1] = (outputs[:,:,1] - outputs[:,:,3]/2)
box_corner[:,:,2] = (outputs[:,:,0] + outputs[:,:,2]/2)
box_corner[:,:,3] = (outputs[:,:,1] + outputs[:,:,3]/2)
# replace in the prediction tensor
outputs[:,:,:4] = box_corner[:,:,:4]

In [100]:
image_pred = outputs[0] # for each image in batch

In [101]:
image_pred.shape

torch.Size([10647, 85])

In [102]:
image_pred[:,5:5+80].shape

torch.Size([10647, 80])

In [103]:
torch.max(torch.tensor([1, 2, 3, 4, 5]), 0)

torch.return_types.max(
values=tensor(5),
indices=tensor(4))

In [104]:
max_conf, max_conf_idx = torch.max(image_pred[:,5:5+80], 1)

In [105]:
max_conf.shape

torch.Size([10647])

In [106]:
max_conf = max_conf.float().unsqueeze(1) # max_conf (1, 1, 85)
max_conf_score = max_conf_idx.float().unsqueeze(1) # max_conf_score (1, 1, 85)

In [107]:
image_pred[:,:5].shape

torch.Size([10647, 5])

In [108]:
max_conf.shape

torch.Size([10647, 1])

In [109]:
max_conf_score.shape

torch.Size([10647, 1])

In [110]:
seq = (image_pred[:,:5], max_conf, max_conf_score)
image_pred = torch.cat(seq, 1)
image_pred.shape

torch.Size([10647, 7])

In [111]:
image_pred[:,4]

tensor([0., 0., 0.,  ..., 0., 0., 0.])

In [112]:
non_zero_idx = (torch.nonzero(image_pred[:, 4]))

In [113]:
non_zero_idx.shape

torch.Size([1, 1])

In [114]:
image_pred_ = image_pred[non_zero_idx.squeeze(), :].view(-1,7)

In [115]:
image_pred_

tensor([[-17.8275, 275.9133, 422.6099, 319.4225,   0.9276,   0.9997,   2.0000]])

In [122]:
conf_sort_idx = torch.sort(image_pred_[:,4], descending=True)[1]
image_pred_ordered = image_pred_[conf_sort_idx]

In [123]:
nms_conf = 0.4

conf_sort_idx = torch.sort(image_pred_[:,4], descending=True)[1]
image_pred_ordered = image_pred_[conf_sort_idx]

n_dets = image_pred_ordered.shape[0]
for idx in range(n_dets):
    # print(idx)
    # print(pred)
    try:
        ious = bbox_iou(image_pred_ordered[idx].unsqueeze(0), image_pred_ordered[idx+1:])
    except ValueError:
        break

    except IndexError:
        break

    iou_mask = (ious < nms_conf).float().unsqueeze(1)
    image_pred_ordered[idx+1:] *= iou_mask

    non_zero_idx = torch.nonzero(image_pred_ordered[:,4]).squeeze()
    image_pred_ordered = image_pred_ordered[non_zero_idx].view(-1,7)
    # print(image_pred_ordered)
    # print(image_pred_ordered.shape)

In [124]:
image_pred_ordered

tensor([[-17.8275, 275.9133, 422.6099, 319.4225,   0.9276,   0.9997,   2.0000]])

In [125]:
batch_idx = image_pred_ordered.new(image_pred_ordered.size(0), 1).fill_(idx)
batch_idx

tensor([[0.]])

In [ ]:
seq = batch_idx, image_pred_ordered
output = torch.cat(seq,1)
output

tensor([[  5.0000, 160.9702, 157.9619, 292.7193, 235.9922,   0.9994,   0.9109,
           2.0000],
        [  5.0000,  19.1932, 172.2979, 118.8749, 245.4051,   0.9974,   0.9965,
           2.0000],
        [  5.0000, -27.2960, 272.9604, 435.2546, 322.3747,   0.9865,   1.0000,
           2.0000],
        [  5.0000, 364.7225, 136.0251, 377.4218, 160.3831,   0.8234,   0.9997,
           9.0000],
        [  5.0000, 374.9868, 135.9671, 390.3168, 160.1495,   0.5790,   0.9998,
           9.0000]])